## BERT Double on ToS dataset

##### Code Repository Reference - https://github.com/reglab/casehold
##### Dataset for ToS - http://claudette.eui.eu/ToS.zip

##### Step 1 : Download the ToS dataset, store Labels and Sentences folder it in data folder and execute the following command blocks

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [2]:
all_sentences_files = os.listdir("./data/Sentences/")

In [3]:
prefix_sentence = "./data/Sentences/"
prefix_label = "./data/Labels/"

In [4]:
clauses = []
for file in all_sentences_files:
    sentence_file_path = prefix_sentence + file 
    label_file_path = prefix_label + file
    sentences_df = pd.read_csv(sentence_file_path, sep="dummy_separator", header=None)
    sentences_df.columns = ["sentences"]
    label_df = pd.read_csv(label_file_path, sep=" ", header=None)
    label_df.columns = ["label"]
    label_df["label_converted"] = np.where(label_df["label"] == -1, 0, 1)
    df_concat = pd.concat([label_df["label_converted"], sentences_df], axis=1)
    clauses.append(df_concat)

In [5]:
colnames = ["label_converted", "sentences"]
clauses_df = pd.DataFrame(columns = colnames)
for df in clauses:
    clauses_df = clauses_df.append(df)

In [6]:
clauses_df.rename(columns={'label_converted': 'label', 'sentences': 'sentences'}, inplace=True)

In [7]:
clauses_df.to_csv("./data/tos_clauses.csv", index=False)

##### Step 2: Compute per example average pretrain loss

In [8]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [9]:
!python3 classification/run_glue.py \
  --model_name_or_path zlucia/bert-double \
  --train_file data/tos_clauses.csv \
  --validation_file data/tos_clauses.csv \
  --ptl=True \
  --max_seq_length 128 \
  --output_dir logs/tos/bertdouble \
  --overwrite_output_dir

10/13/2022 18:18:37 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
10/13/2022 18:18:37 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False

[INFO|configuration_utils.py:653] 2022-10-13 18:18:39,100 >> loading configuration file config.json from cache at /Users/shreyaprabhu/.cache/huggingface/hub/models--zlucia--bert-double/snapshots/0ce1a5b13ad6781ac7784521d1b56e1f48c89cf7/config.json
[INFO|configuration_utils.py:705] 2022-10-13 18:18:39,101 >> Model config BertConfig {
  "_name_or_path": "zlucia/bert-double",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_

##### Step 3: Split data into train and dev. Fine tune on train dataset and evaluate performance on dev dataset. Logs andPredictions are directed to output directory logs/tos/bertdouble

In [10]:
train, dev = train_test_split(clauses_df, test_size=0.2, random_state=42, stratify=clauses_df[['label']])

In [11]:
train.to_csv("./data/tos_clauses_train.csv", index=False)
dev.to_csv("./data/tos_clauses_dev.csv", index=False)

In [12]:
# Download model from Hugging Face model repository
!python3 classification/run_glue.py \
  --model_name_or_path zlucia/bert-double \
  --train_file data/tos_clauses_train.csv \
  --validation_file data/tos_clauses_dev.csv \
  --do_train \
  --do_eval \
  --evaluation_strategy steps \
  --max_seq_length 128 \
  --per_device_train_batch_size=16 \
  --learning_rate=1e-5 \
  --num_train_epochs=2.0 \
  --output_dir logs/tos/bertdouble \
  --overwrite_output_dir \
  --logging_steps 50

10/13/2022 18:26:30 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
10/13/2022 18:26:30 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=50,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,

[INFO|configuration_utils.py:653] 2022-10-13 18:26:33,068 >> loading configuration file config.json from cache at /Users/shreyaprabhu/.cache/huggingface/hub/models--zlucia--bert-double/snapshots/0ce1a5b13ad6781ac7784521d1b56e1f48c89cf7/config.json
[INFO|configuration_utils.py:705] 2022-10-13 18:26:33,069 >> Model config BertConfig {
  "_name_or_path": "zlucia/bert-double",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_

{'loss': 0.4588, 'learning_rate': 9.469214437367304e-06, 'epoch': 0.11}         
  5%|██▏                                       | 50/942 [02:18<39:09,  2.63s/it][INFO|trainer.py:725] 2022-10-13 18:28:53,312 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentences. If sentences are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:2907] 2022-10-13 18:28:53,313 >> ***** Running Evaluation *****
[INFO|trainer.py:2909] 2022-10-13 18:28:53,313 >>   Num examples = 1883
[INFO|trainer.py:2912] 2022-10-13 18:28:53,313 >>   Batch size = 8

 38%|███████████████▊                          | 89/236 [00:35<00:59,  2.47it/s]


 79%|████████████████████████████████▎        | 186/236 [01:15<00:21,  2.34it/s]


100%|█████████████████████████████████████████| 236/236 [01:35<00:00,  2.87it/s]10/13/2022 18:30:30 - INFO - /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/datasets/metric.py -   Removing /Users/shreyaprabhu/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow
                                                                                
{'eval_loss': 0.35425686836242676, 'eval_f1': 0.0, 'eval_runtime': 97.2981, 'eval_samples_per_second': 19.353, 'eval_steps_per_second': 2.426, 'epoch': 0.11}
100%|█████████████████████████████████████████| 236/236 [01:36<00:00,  2.87it/s]
{'loss': 0.3132, 'learning_rate': 8.938428874734608e-06, 'epoch': 0.21}         
 11%|████▎                                    | 100/942 [06:11<39:27,  2.81s/it][INFO|trainer.py:725] 2022-10-13 18:32:46,436 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentences. If sent

...

 58%|███████████████████████▋                 | 136/236 [01:01<00:43,  2.30it/s]


 99%|████████████████████████████████████████▍| 233/236 [01:44<00:01,  2.14it/s]


100%|█████████████████████████████████████████| 236/236 [01:45<00:00,  2.68it/s]10/13/2022 20:10:58 - INFO - /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/datasets/metric.py -   Removing /Users/shreyaprabhu/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow
                                                                                
{'eval_loss': 0.18179234862327576, 'eval_f1': 0.7607655502392345, 'eval_runtime': 106.407, 'eval_samples_per_second': 17.696, 'eval_steps_per_second': 2.218, 'epoch': 1.8}
100%|█████████████████████████████████████████| 236/236 [01:45<00:00,  2.68it/s]
{'loss': 0.171, 'learning_rate': 4.45859872611465e-07, 'epoch': 1.91}           
 96%|█████████████████████████████████████▎ | 900/942 [1:46:33<01:48,  2.58s/it][INFO|trainer.py:725] 2022-10-13 20:13:08,801 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sent

 74%|██████████████████████████████▍          | 175/236 [01:19<00:27,  2.19it/s]


100%|█████████████████████████████████████████| 236/236 [01:47<00:00,  2.60it/s]10/13/2022 20:14:57 - INFO - /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/datasets/metric.py -   Removing /Users/shreyaprabhu/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow
                                                                                
{'eval_loss': 0.17678211629390717, 'eval_f1': 0.760204081632653, 'eval_runtime': 108.605, 'eval_samples_per_second': 17.338, 'eval_steps_per_second': 2.173, 'epoch': 1.91}
100%|███████████████████████████████████████| 942/942 [1:50:12<00:00,  2.37s/it][INFO|trainer.py:1852] 2022-10-13 20:16:47,671 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


{'train_runtime': 6612.8468, 'train_samples_per_second': 2.278, 'train_steps_per_second': 0.142, 'train_loss': 0.18250087061758508, 'epoch': 2.0}
100%|███████████████████████████████████████| 942/942 [1:50:12<00:00,  7.02s/it